## Python Connector using Apache Arrow Format
Fetching result sets using the Snowflake Python Connector now leverages the Arrow columnar format,
This avoids the overhead previously associated with serializing and deserializing Snowflake data 
structures which are also in columnar format.

With Snowflake Connector 2.2.0 (or higher), when you work with Pandas DataFrames, the performance is even better with the introduction of our new Python APIs, which download result sets directly into a Pandas DataFrame.

In [ ]:
import snowflake.connector
import pandas

In [ ]:
import getpass

account = input('Snowflake Account Name: ')
user =  input('Snowflake User Name: ')
password = getpass.getpass('Snowflake Password: ')
warehouse = user + "_WH";  # [login]_wh

con = snowflake.connector.connect(
  user=user,
  password=password,
  account=account
)
cur = con.cursor()

## Python APIs for Pandas

Fetching result sets leveraging the Arrow columnar format

In [ ]:
# Test table is TPCH_SF1.LINEITEM which has 6M rows.
# Its compressed size in Snowflake is 165 MB.
# Fetch the result set from the cursor and deliver it as the Pandas DataFrame.
sql = "select * from SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.LINEITEM"

`fetch_pandas_all()` method fetches all the rows in a cursor and loads them into a Pandas DataFrame.

In [ ]:
%%time
cur.execute(sql);

# Fetch the result set from the cursor and deliver it as the Pandas DataFrame.
df = cur.fetch_pandas_all()
# Number of rows
print(len(df.index))

In [ ]:
# Test table is TPCH LINEITEM which has 60M rows.
# Its compressed size in Snowflake is 1.6 GB.
# Fetch the result set from the cursor and deliver it as the Pandas DataFrame.
sql = "select * from SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.LINEITEM"

`fetch_pandas_batches` This method fetches a subset of the rows in a cursor and delivers them to a Pandas DataFrame.

Depending upon the number of rows in the result set, as well as the number of rows specified in the method call, the method might need to be called more than once, or it might return all rows in a single batch if they all fit.

In [ ]:
%%time
cur.execute(sql);

# Fetch the result set from the cursor and deliver it as the Pandas DataFrame.
number_of_rows = 0
for df in cur.fetch_pandas_batches():
    index = df.index
    number_of_rows += len(index)
print(number_of_rows)